# Imports

In [1]:
import ccxt
import pandas as pd
import time
import numpy as np
import talib as ta
from platypus import NSGAII, Problem, Real
import random
import backtrader as bt


# Data download

In [2]:
def download_crypto_data(symbol, timeframe='1d', limit=1000):
    exchange = ccxt.binance()  # You can use other exchanges
    since = exchange.parse8601('2021-01-01T00:00:00Z')
    data = []
    
    while since < exchange.milliseconds():
        try:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit)
            if len(ohlcv) == 0:
                break
            data += ohlcv
            since = ohlcv[-1][0] + 1
            time.sleep(exchange.rateLimit / 1000)  # Be mindful of rate limits
        except Exception as e:
            print(f"Error fetching data: {e}")
            break
    
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

# Example Usage
btc_data = download_crypto_data('BTC/USDT')
eth_data = download_crypto_data('ETH/USDT')

In [3]:
btc_data.tail()
eth_data.tail()

,open,high,low,close,volume
timestamp,,,,,
2024-09-03,2538.00,2553.60,2411.12,2425.29,224793.1299
2024-09-04,2425.28,2490.00,2306.65,2450.71,384179.3854
2024-09-05,2450.71,2466.00,2348.04,2368.81,278549.0969
2024-09-06,2368.81,2408.83,2150.55,2225.23,643027.1110
2024-09-07,2225.24,2311.27,2220.98,2286.75,167920.3152


# Alphas

## Helpers

In [4]:
# Helper functions
def rank(series):
    return series.rank(pct=True)

def ts_rank(series, window):
    return series.rolling(window).apply(lambda x: rank(x).iloc[-1])

def ts_argmax(series, window):
    return series.rolling(window).apply(np.argmax) + 1

def correlation(x, y, window):
    return x.rolling(window).corr(y)

## Alphas

In [5]:

# Alpha 1
def alpha_1(data):
    returns = data['close'].pct_change()
    cond = np.where(returns < 0, ta.STDDEV(returns, timeperiod=20), data['close'])
    return rank(ts_argmax(np.power(cond, 2), 5)) - 0.5

# Alpha 2
def alpha_2(data):
    return -1 * correlation(rank(np.log(data['volume']).diff(2)), rank((data['close'] - data['open']) / data['open']), 6)

# Alpha 3
def alpha_3(data):
    return -1 * correlation(rank(data['open']), rank(data['volume']), 10)

# Alpha 4
def alpha_4(data):
    return -1 * ts_rank(rank(data['low']), 9)

# Alpha 5
def alpha_5(data):
    vwap = (data['close'] * data['volume']).rolling(10).sum() / data['volume'].rolling(10).sum()
    return rank(data['open'] - vwap) * (-1 * abs(rank(data['close'] - vwap)))

# Alpha 6
def alpha_6(data):
    return -1 * correlation(data['open'], data['volume'], 10)

# MOEA

In [6]:
# Define the problem
class AlphaOptimization(Problem):
    def __init__(self, data, num_alphas=6):
        super(AlphaOptimization, self).__init__(num_alphas, 2)  # 2 objectives: Sharpe and Volatility
        self.data = data
        self.types[:] = [Real(0, 1) for _ in range(num_alphas)]  # Weights for each alpha


    def evaluate(self, solution):
        weights = solution.variables
        portfolio_returns = sum(weights[i] * self.data[f'alpha_{i+1}'] for i in range(len(weights)))
        sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std()
        volatility = portfolio_returns.std()
        solution.objectives[:] = [-sharpe_ratio, volatility]  # Minimize volatility, maximize Sharpe

# Example usage
alpha_data = pd.DataFrame()  # Suppose we have alpha signals as columns

problem = AlphaOptimization(alpha_data)
algorithm = NSGAII(problem)
algorithm.run(10000)

# Retrieve results
best_solution = min(algorithm.result, key=lambda s: s.objectives[0])  # Maximize Sharpe Ratio
print("Best Weights:", best_solution.variables)

KeyError: 'alpha_1'

# Backtesting Using backtrader

In [ ]:

class AlphaStrategy(bt.Strategy):
    def __init__(self, alphas):
        self.alphas = alphas

    def next(self):
        weights = self.alphas  # Use optimized weights from MOEA
        alpha_combination = sum(weights[i] * self.data.lines[i] for i in range(len(weights)))
        if alpha_combination > 0:
            self.buy()
        elif alpha_combination < 0:
            self.sell()

# Backtest with the data
cerebro = bt.Cerebro()
cerebro.addstrategy(AlphaStrategy, alphas=best_solution.variables)
data = bt.feeds.PandasData(dataname=btc_data)
cerebro.adddata(data)
cerebro.run()
cerebro.plot()
